# 만들어진 Word2Vec model을 통한 감정분석 실시
> * Positive or Negative

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os

os.environ['KERAS_BACKEND']='tensorflow'

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot


import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\pc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print (device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10419045285149508714
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4984510873
locality {
  bus_id: 1
}
incarnation: 4320047186308386534
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:09:00.0, compute capability: 6.1"
]


In [3]:
def Make_Roc_Curve(x, y, model1, model2, model3, model4):
    print ('Logistic Regression')
    fpr1, tpr1, thresholds1 = roc_curve(y, model1.predict(x))
    print ('Random Forest')
    fpr2, tpr2, thresholds2 = roc_curve(y, model2.predict(x))
    print ('Kernel SVM')
    fpr3, tpr3, thresholds3 = roc_curve(y, model3.predict(x))
    print ('XGBoost')
    import xgboost as xgb
    fpr4, tpr4, thresholds4 = roc_curve(y, model4.predict(xgb.DMatrix(x)))
    plt.plot(fpr1, tpr1, label="Logistic Regression")
    plt.plot(fpr2, tpr2, label="RandomForest")
    plt.plot(fpr3, tpr3, label="Kernel SVM")
    plt.plot(fpr4, tpr4, label='XGBoost')
    plt.legend()
    plt.plot([0, 1], [0, 1], 'k--', label="random guess")
    plt.xlabel('False Positive Rate (Fall-Out)')
    plt.ylabel('True Positive Rate (Recall)')
    plt.title('Receiver operating characteristic example')
    plt.show()

In [4]:
def plot_history(history):
    """Plot model history after `fit()`.
    """

    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()

In [5]:
def Make_TSNE1(n_component, model, wv, limit):
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    import pandas as pd
    from tqdm import tqdm
    tqdm.pandas(desc="progress-bar")
    wv = wv[:limit]
    tsne_model = TSNE(n_components=n_component,
                       verbose = 1, random_state = 0)
    tsne_w2v = tsne_model.fit_transform(wv)
    tsne_df = pd.DataFrame(tsne_w2v, columns = ['x', 'y'])
    tsne_df['words'] = list(model.wv.vocab.keys())[:limit]
    i = 0
    for i in tqdm(range(tsne_df['words'].size)):
        plt.scatter(tsne_df['x'][i], tsne_df['y'][i])
        plt.annotate(tsne_df['words'][i], 
                xy = (tsne_df['x'][i], tsne_df['y'][i]))
    plt.show()

In [6]:
def Make_TSNE2(n_component, model, wv, limit):
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    import pandas as pd
    from tqdm import tqdm
    import bokeh.plotting as bp
    from bokeh.models import HoverTool, BoxSelectTool
    from bokeh.plotting import figure, show, output_notebook
    
    output_notebook()
    plot_tfidf = bp.figure(plot_width=500, plot_height=500, title="A map of word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

    word_vectors = [model[w] for w in tqdm(list(model.wv.vocab.keys())[:limit])]

    tsne_model = TSNE(n_components=n_component, verbose=1, random_state=0)
    tsne_w2v = tsne_model.fit_transform(word_vectors)
    # putting everything in a dataframe
    tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
    tsne_df['words'] = list(model.wv.vocab.keys())[:limit]

    # plotting. the corresponding word appears when you hover on the data point.
    plot_tfidf.scatter(x='x', y='y', source=tsne_df)
    hover = plot_tfidf.select(dict(type=HoverTool))
    hover.tooltips={"word": "@words"}
    show(plot_tfidf)

In [7]:
def Build_tfidf(data):
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(analyzer = lambda x: x, min_df = 2)
    matrix = vectorizer.fit_transform([x.words for x in tqdm(data)])
    print (matrix.shape)
    tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
    print ('vocab size : {}'.format(len(tfidf)))    
    return tfidf

In [8]:
def buildWordVector(tokens, model, size, tfidf):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [9]:
def Make_Pre_Data(model, tfidf, size, train, test):
    from datetime import datetime
    start = datetime.now()
    print (str(model))
    wv = [model[w] for w in tqdm(model.wv.vocab.keys())]
    process1 = datetime.now()
    print ('running time : {}'.format(process1 - start))
    
    print ('Vectorizing Train Data')
    train_vecs_w2v = np.concatenate([buildWordVector(z, model, size, tfidf) for z in tqdm(map(lambda x: x.words, train))])
    print ('scaling Train Data')
    train_vecs_w2v = scale(train_vecs_w2v)
    process2 = datetime.now()
    print ('running time : {}'.format(process2 - process1))
    
    print ('Vectorizing Test Data')
    test_vecs_w2v = np.concatenate([buildWordVector(z, model, size, tfidf) for z in tqdm(map(lambda x: x.words, test))])
    print ('scaling Test Data')
    test_vecs_w2v = scale(test_vecs_w2v)
    process3 = datetime.now()
    print ('running time : {}'.format(process3 - process2))
    
    print ('total running time : {}'.format(process3 - start))
    return wv, train_vecs_w2v, test_vecs_w2v

In [10]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    #loadModelPath = './model/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
saveTrainPath = './data/pre_data/train_test_Data2/'
saveClassifierPath = './data/pre_data/classifier/'

In [11]:
def ReMake_Outcome(train_y, test_y):
    from tqdm import tqdm
    tqdm.pandas(desc="progress-bar") 
    train_y = [y[0] for y in tqdm(train_y)]
    test_y = [y[0] for y in tqdm(test_y)]
    return train_y, test_y

In [12]:
def Return_ModelName(model, tagger):
    size = model.vector_size
    epochs = model.epochs
    window = model.window
    negative = model.negative 
    hs = model.hs 
    sg = model.sg 
    cbow_mean = model.cbow_mean 
    min_count = model.min_count 
    min_alpha = model.min_alpha
    alpha = model.alpha
    modelName = 'word2vec_size-{}_epochs-{}_window-{}_negative-{}_hs-{}_sg-{}_cbow_mean-{}_min_count-{}_min_alpha-{}_alpha-{}_by-{}'.format(
    size, epochs, window, negative, hs, sg, cbow_mean, min_count, 
    min_alpha, alpha, tagger)
    return modelName

In [13]:
cores = int(multiprocessing.cpu_count() )
print (cores)

12


In [14]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Twitter

###  Load Word2Vec model을 만들기 위해 사용한 사용된 train set

In [15]:
train = pickle.load(open('./data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_ct.pickled','rb'))
y_train = np.array([doc.sentiment for doc in tqdm(train)])

100%|██████████| 442359/442359 [00:00<00:00, 1512694.70it/s]


### train set을 사용하여 Tf-Idf vectorizer을 만듦

In [16]:
tfidf = Build_tfidf(train)

100%|██████████| 442359/442359 [00:00<00:00, 1502356.81it/s]


(442359, 159113)
vocab size : 159113


### Load Word2Vec model을 만들기 위해 사용한 사용된 testset

In [17]:
test = pickle.load(open('./data/pre_data/train_test_Data/pre_data_test_for_word2vec_sentiment_by_ct.pickled','rb'))
y_test = np.array([doc.sentiment for doc in tqdm(test)])

100%|██████████| 49151/49151 [00:00<00:00, 1321425.28it/s]


In [18]:
train_y2, test_y2 = ReMake_Outcome(y_train, y_test)

100%|██████████| 49151/49151 [00:00<00:00, 1066675.48it/s]


### Load Model

In [ ]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

### Model_1

In [ ]:
wv1, train_vecs_w2v, test_vecs_w2v = Make_Pre_Data(model1, tfidf, 1000, train, test)
modelName = Return_ModelName(model1,'ct')

#### t-SNE
> * t-분포 확률적 임베딩
> * 데이터의 차원 축소에 사용되는 기계 학습 알고리즘
> * 비선형 차원 축소 기법으로 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용
> * 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑
##### word : 10000

In [ ]:
%%time
Make_TSNE2(2, model1, wv1, 10000)

#### 분류모델 : Logistic Regression

In [ ]:
%%time
classifier = LogisticRegression(max_iter = 250, n_jobs = cores)
classifier.fit(train_vecs_w2v, train_y2)
print (classifier.get_params())
print( 'score : {}'.format(classifier.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier.predict(test_vecs_w2v)))
pickle.dump(classifier,open(saveClassifierPath+'LogisticRegression_'+modelName, 'wb'))

#### 분류모델 : Random Forest

In [ ]:
%%time
classifier2 = RandomForestClassifier(n_estimators = 75, n_jobs = cores)
classifier2.fit(train_vecs_w2v, train_y2)
print (classifier2.get_params())
print( 'score : {}'.format(classifier2.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier2.predict(test_vecs_w2v)))
pickle.dump(classifier2,open(saveClassifierPath+'RandomForestClassifier_'+modelName, 'wb'))

#### 분류모델 : C - Support Vector Classification

In [ ]:
%%time
scaling = MinMaxScaler(feature_range=(-1, 1)).fit(train_vecs_w2v)
train_vecs_w2v2 = scaling.transform(train_vecs_w2v)
test_vecs_w2v2 = scaling.transform(test_vecs_w2v)
classifier3 =  SVC(kernel = 'linear',
        cache_size= 1024, max_iter = 1500, verbose = True) 
classifier3.fit(train_vecs_w2v2, train_y2)
print (classifier3.get_params())
print( 'score : {}'.format(classifier3.score(test_vecs_w2v2, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier3.predict(test_vecs_w2v2)))
pickle.dump(classifier3,open(saveClassifierPath+'SVC_'+modelName, 'wb'))

#### 분류모델 : XGBOOST

In [ ]:
max_depth = 5
subsample = 0.7
colsample_bytree= 0.7
params ={
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "max_depth" : max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent":1,
    'eta':0.125,
    'nthread' : cores
    
    }
num_boost_round = 200
early_stopping_rounds = 10
test_size = 0.15

In [ ]:
dtrain = xgb.DMatrix(train_vecs_w2v, y_train)
dvalid = xgb.DMatrix(test_vecs_w2v, y_test)

In [ ]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals = watchlist, early_stopping_rounds = early_stopping_rounds, verbose_eval = True)
test_prediction = gbm.predict(xgb.DMatrix(test_vecs_w2v))
test_class = np.round(test_prediction)
print (accuracy_score(y_test, test_class))
print (classification_report(y_test, test_class))
gbm.save_model(saveClassifierPath+'XGBoost_'+modelName)

In [ ]:
%%time
Make_Roc_Curve(test_vecs_w2v, y_test, classifier, classifier2, classifier3, gbm)

In [ ]:
del classifier
del classifier2
del classifier3
del gbm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=1000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0,
          validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])

score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_1_'+modelName)

In [ ]:
plot_history(history)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0, validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])
score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_2_'+modelName)

In [ ]:
plot_history(history)

### Model_2

In [ ]:
wv1, train_vecs_w2v, test_vecs_w2v = Make_Pre_Data(model2, tfidf, 1000, train, test)
modelName = Return_ModelName(model2,'ct')

#### t-SNE
> * t-분포 확률적 임베딩
> * 데이터의 차원 축소에 사용되는 기계 학습 알고리즘
> * 비선형 차원 축소 기법으로 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용
> * 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑
##### word : 10000

In [ ]:
%%time
Make_TSNE2(2, model2, wv1, 10000)

#### 분류모델 : Logistic Regression

In [ ]:
%%time
classifier = LogisticRegression(max_iter = 250, n_jobs = cores)
classifier.fit(train_vecs_w2v, train_y2)
print (classifier.get_params())
print( 'score : {}'.format(classifier.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier.predict(test_vecs_w2v)))
pickle.dump(classifier,open(saveClassifierPath+'LogisticRegression_'+modelName, 'wb'))

#### 분류모델 : Random Forest

In [ ]:
%%time
classifier2 = RandomForestClassifier(n_estimators = 75, n_jobs = cores)
classifier2.fit(train_vecs_w2v, train_y2)
print (classifier2.get_params())
print( 'score : {}'.format(classifier2.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier2.predict(test_vecs_w2v)))
pickle.dump(classifier2,open(saveClassifierPath+'RandomForestClassifier_'+modelName, 'wb'))

#### 분류모델 : C - Support Vector Classification

In [ ]:
%%time
scaling = MinMaxScaler(feature_range=(-1, 1)).fit(train_vecs_w2v)
train_vecs_w2v2 = scaling.transform(train_vecs_w2v)
test_vecs_w2v2 = scaling.transform(test_vecs_w2v)
classifier3 =  SVC(kernel = 'linear',
        cache_size= 1024, max_iter = 1500, verbose = True) 
classifier3.fit(train_vecs_w2v2, train_y2)
print (classifier3.get_params())
print( 'score : {}'.format(classifier3.score(test_vecs_w2v2, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier3.predict(test_vecs_w2v2)))
pickle.dump(classifier3,open(saveClassifierPath+'SVC_'+modelName, 'wb'))

#### 분류모델 : XGBOOST

In [ ]:
max_depth = 5
subsample = 0.7
colsample_bytree= 0.7
params ={
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "max_depth" : max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent":1,
    'eta':0.125,
    'nthread' : cores
    
    }
num_boost_round = 200
early_stopping_rounds = 10
test_size = 0.15

In [ ]:
dtrain = xgb.DMatrix(train_vecs_w2v, y_train)
dvalid = xgb.DMatrix(test_vecs_w2v, y_test)

In [ ]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals = watchlist, early_stopping_rounds = early_stopping_rounds, verbose_eval = True)
test_prediction = gbm.predict(xgb.DMatrix(test_vecs_w2v))
test_class = np.round(test_prediction)
print (accuracy_score(y_test, test_class))
print (classification_report(y_test, test_class))
gbm.save_model(saveClassifierPath+'XGBoost_'+modelName)

In [ ]:
%%time
Make_Roc_Curve(test_vecs_w2v, y_test, classifier, classifier2, classifier3, gbm)

In [ ]:
del classifier
del classifier2
del classifier3
del gbm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=1000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0,
          validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])

score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_1_'+modelName)

In [ ]:
plot_history(history)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0, validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])
score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_2_'+modelName)

In [ ]:
plot_history(history)

### Model_3

In [ ]:
wv1, train_vecs_w2v, test_vecs_w2v = Make_Pre_Data(model3, tfidf, 1000, train, test)
modelName = Return_ModelName(model3,'ct')

#### t-SNE
> * t-분포 확률적 임베딩
> * 데이터의 차원 축소에 사용되는 기계 학습 알고리즘
> * 비선형 차원 축소 기법으로 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용
> * 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑
##### word : 10000

In [ ]:
%%time
Make_TSNE2(2, model3, wv1, 10000)

#### 분류모델 : Logistic Regression

In [ ]:
%%time
classifier = LogisticRegression(max_iter = 250, n_jobs = cores)
classifier.fit(train_vecs_w2v, train_y2)
print (classifier.get_params())
print( 'score : {}'.format(classifier.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier.predict(test_vecs_w2v)))
pickle.dump(classifier,open(saveClassifierPath+'LogisticRegression_'+modelName, 'wb'))

#### 분류모델 : Random Forest

In [ ]:
%%time
classifier2 = RandomForestClassifier(n_estimators = 75, n_jobs = cores)
classifier2.fit(train_vecs_w2v, train_y2)
print (classifier2.get_params())
print( 'score : {}'.format(classifier2.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier2.predict(test_vecs_w2v)))
pickle.dump(classifier2,open(saveClassifierPath+'RandomForestClassifier_'+modelName, 'wb'))

#### 분류모델 : C - Support Vector Classification

In [ ]:
%%time
scaling = MinMaxScaler(feature_range=(-1, 1)).fit(train_vecs_w2v)
train_vecs_w2v2 = scaling.transform(train_vecs_w2v)
test_vecs_w2v2 = scaling.transform(test_vecs_w2v)
classifier3 =  SVC(kernel = 'linear',
        cache_size= 1024, max_iter = 1500, verbose = True) 
classifier3.fit(train_vecs_w2v2, train_y2)
print (classifier3.get_params())
print( 'score : {}'.format(classifier3.score(test_vecs_w2v2, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier3.predict(test_vecs_w2v2)))
pickle.dump(classifier3,open(saveClassifierPath+'SVC_'+modelName, 'wb'))

#### 분류모델 : XGBOOST

In [ ]:
max_depth = 5
subsample = 0.7
colsample_bytree= 0.7
params ={
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "max_depth" : max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent":1,
    'eta':0.125,
    'nthread' : cores
    
    }
num_boost_round = 200
early_stopping_rounds = 10
test_size = 0.15

In [ ]:
dtrain = xgb.DMatrix(train_vecs_w2v, y_train)
dvalid = xgb.DMatrix(test_vecs_w2v, y_test)

In [ ]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals = watchlist, early_stopping_rounds = early_stopping_rounds, verbose_eval = True)
test_prediction = gbm.predict(xgb.DMatrix(test_vecs_w2v))
test_class = np.round(test_prediction)
print (accuracy_score(y_test, test_class))
print (classification_report(y_test, test_class))
gbm.save_model(saveClassifierPath+'XGBoost_'+modelName)

In [ ]:
%%time
Make_Roc_Curve(test_vecs_w2v, y_test, classifier, classifier2, classifier3, gbm)

In [ ]:
del classifier
del classifier2
del classifier3
del gbm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=1000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0,
          validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])

score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_1_'+modelName)

In [ ]:
plot_history(history)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0, validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])
score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_2_'+modelName)

In [ ]:
plot_history(history)

## Mecab

###  Load Word2Vec model을 만들기 위해 사용한 사용된 train set

In [ ]:
train = pickle.load(open('./data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled','rb'))
y_train = np.array([doc.sentiment for doc in tqdm(train)])

### train set을 사용하여 Tf-Idf vectorizer을 만듦

In [ ]:
tfidf = Build_tfidf(train)

### Load Word2Vec model을 만들기 위해 사용한 사용된 testset

In [ ]:
test = pickle.load(open('./data/pre_data/train_test_Data/pre_data_test_for_word2vec_sentiment_by_mecab.pickled','rb'))
y_test = np.array([doc.sentiment for doc in tqdm(test)])

In [ ]:
train_y2, test_y2 = ReMake_Outcome(y_train, y_test)

### Load Model

In [ ]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-mecab.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-mecab.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-mecab.model')

### Model_1 

In [ ]:
wv1, train_vecs_w2v, test_vecs_w2v = Make_Pre_Data(model1, tfidf, 1000, train, test)
modelName = Return_ModelName(model1,'mecab')

#### t-SNE
> * t-분포 확률적 임베딩
> * 데이터의 차원 축소에 사용되는 기계 학습 알고리즘
> * 비선형 차원 축소 기법으로 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용
> * 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑
##### word : 10000

In [ ]:
%%time
Make_TSNE2(2, model1, wv1, 10000)

#### 분류모델 : Logistic Regression

In [ ]:
%%time
classifier = LogisticRegression(max_iter = 250, n_jobs = cores)
classifier.fit(train_vecs_w2v, train_y2)
print (classifier.get_params())
print( 'score : {}'.format(classifier.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier.predict(test_vecs_w2v)))
pickle.dump(classifier,open(saveClassifierPath+'LogisticRegression_'+modelName, 'wb'))

#### 분류모델 : Random Forest

In [ ]:
%%time
classifier2 = RandomForestClassifier(n_estimators = 75, n_jobs = cores)
classifier2.fit(train_vecs_w2v, train_y2)
print (classifier2.get_params())
print( 'score : {}'.format(classifier2.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier2.predict(test_vecs_w2v)))
pickle.dump(classifier2,open(saveClassifierPath+'RandomForestClassifier_'+modelName, 'wb'))

#### 분류모델 : C - Support Vector Classification

In [ ]:
%%time
scaling = MinMaxScaler(feature_range=(-1, 1)).fit(train_vecs_w2v)
train_vecs_w2v2 = scaling.transform(train_vecs_w2v)
test_vecs_w2v2 = scaling.transform(test_vecs_w2v)
classifier3 =  SVC(kernel = 'linear',
        cache_size= 1024, max_iter = 1500, verbose = True) 
classifier3.fit(train_vecs_w2v2, train_y2)
print (classifier3.get_params())
print( 'score : {}'.format(classifier3.score(test_vecs_w2v2, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier3.predict(test_vecs_w2v2)))
pickle.dump(classifier3,open(saveClassifierPath+'SVC_'+modelName, 'wb'))

#### 분류모델 : XGBOOST

In [ ]:
max_depth = 5
subsample = 0.7
colsample_bytree= 0.7
params ={
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "max_depth" : max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent":1,
    'eta':0.125,
    'nthread' : cores
    
    }
num_boost_round = 200
early_stopping_rounds = 10
test_size = 0.15

In [ ]:
dtrain = xgb.DMatrix(train_vecs_w2v, y_train)
dvalid = xgb.DMatrix(test_vecs_w2v, y_test)

In [ ]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals = watchlist, early_stopping_rounds = early_stopping_rounds, verbose_eval = True)
test_prediction = gbm.predict(xgb.DMatrix(test_vecs_w2v))
test_class = np.round(test_prediction)
print (accuracy_score(y_test, test_class))
print (classification_report(y_test, test_class))
gbm.save_model(saveClassifierPath+'XGBoost_'+modelName)

In [ ]:
%%time
Make_Roc_Curve(test_vecs_w2v, y_test, classifier, classifier2, classifier3, gbm)

In [ ]:
del classifier
del classifier2
del classifier3
del gbm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=1000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0,
          validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])

score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_1_'+modelName)

In [ ]:
plot_history(history)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0, validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])
score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_2_'+modelName)

In [ ]:
plot_history(history)

### Model_2 

In [ ]:
wv1, train_vecs_w2v, test_vecs_w2v = Make_Pre_Data(model2, tfidf, 1000, train, test)
modelName = Return_ModelName(model2,'mecab')

#### t-SNE
> * t-분포 확률적 임베딩
> * 데이터의 차원 축소에 사용되는 기계 학습 알고리즘
> * 비선형 차원 축소 기법으로 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용
> * 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑
##### word : 10000

In [ ]:
%%time
Make_TSNE2(2, model2, wv1, 10000)

#### 분류모델 : Logistic Regression

In [ ]:
%%time
classifier = LogisticRegression(max_iter = 250, n_jobs = cores)
classifier.fit(train_vecs_w2v, train_y2)
print (classifier.get_params())
print( 'score : {}'.format(classifier.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier.predict(test_vecs_w2v)))
pickle.dump(classifier,open(saveClassifierPath+'LogisticRegression_'+modelName, 'wb'))

#### 분류모델 : Random Forest

In [ ]:
%%time
classifier2 = RandomForestClassifier(n_estimators = 75, n_jobs = cores)
classifier2.fit(train_vecs_w2v, train_y2)
print (classifier2.get_params())
print( 'score : {}'.format(classifier2.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier2.predict(test_vecs_w2v)))
pickle.dump(classifier2,open(saveClassifierPath+'RandomForestClassifier_'+modelName, 'wb'))

#### 분류모델 : C - Support Vector Classification

In [ ]:
%%time
scaling = MinMaxScaler(feature_range=(-1, 1)).fit(train_vecs_w2v)
train_vecs_w2v2 = scaling.transform(train_vecs_w2v)
test_vecs_w2v2 = scaling.transform(test_vecs_w2v)
classifier3 =  SVC(kernel = 'linear',
        cache_size= 1024, max_iter = 1500, verbose = True) 
classifier3.fit(train_vecs_w2v2, train_y2)
print (classifier3.get_params())
print( 'score : {}'.format(classifier3.score(test_vecs_w2v2, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier3.predict(test_vecs_w2v2)))
pickle.dump(classifier3,open(saveClassifierPath+'SVC_'+modelName, 'wb'))

#### 분류모델 : XGBOOST

In [ ]:
max_depth = 5
subsample = 0.7
colsample_bytree= 0.7
params ={
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "max_depth" : max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent":1,
    'eta':0.125,
    'nthread' : cores
    
    }
num_boost_round = 200
early_stopping_rounds = 10
test_size = 0.15

In [ ]:
dtrain = xgb.DMatrix(train_vecs_w2v, y_train)
dvalid = xgb.DMatrix(test_vecs_w2v, y_test)

In [ ]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals = watchlist, early_stopping_rounds = early_stopping_rounds, verbose_eval = True)
test_prediction = gbm.predict(xgb.DMatrix(test_vecs_w2v))
test_class = np.round(test_prediction)
print (accuracy_score(y_test, test_class))
print (classification_report(y_test, test_class))
gbm.save_model(saveClassifierPath+'XGBoost_'+modelName)

In [ ]:
%%time
Make_Roc_Curve(test_vecs_w2v, y_test, classifier, classifier2, classifier3, gbm)

In [ ]:
del classifier
del classifier2
del classifier3
del gbm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=1000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0,
          validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])

score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_1_'+modelName)

In [ ]:
plot_history(history)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0, validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])
score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_2_'+modelName)

In [ ]:
plot_history(history)

### Model_3

In [ ]:
wv1, train_vecs_w2v, test_vecs_w2v = Make_Pre_Data(model3, tfidf, 1000, train, test)
modelName = Return_ModelName(model3,'mecab')

#### t-SNE
> * t-분포 확률적 임베딩
> * 데이터의 차원 축소에 사용되는 기계 학습 알고리즘
> * 비선형 차원 축소 기법으로 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용
> * 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑
##### word : 10000

In [ ]:
%%time
Make_TSNE2(2, model3, wv1, 10000)

#### 분류모델 : Logistic Regression

In [ ]:
%%time
classifier = LogisticRegression(max_iter = 250, n_jobs = cores)
classifier.fit(train_vecs_w2v, train_y2)
print (classifier.get_params())
print( 'score : {}'.format(classifier.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier.predict(test_vecs_w2v)))
pickle.dump(classifier,open(saveClassifierPath+'LogisticRegression_'+modelName, 'wb'))

#### 분류모델 : Random Forest

In [ ]:
%%time
classifier2 = RandomForestClassifier(n_estimators = 75, n_jobs = cores)
classifier2.fit(train_vecs_w2v, train_y2)
print (classifier2.get_params())
print( 'score : {}'.format(classifier2.score(test_vecs_w2v, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier2.predict(test_vecs_w2v)))
pickle.dump(classifier2,open(saveClassifierPath+'RandomForestClassifier_'+modelName, 'wb'))

#### 분류모델 : C - Support Vector Classification

In [ ]:
%%time
scaling = MinMaxScaler(feature_range=(-1, 1)).fit(train_vecs_w2v)
train_vecs_w2v2 = scaling.transform(train_vecs_w2v)
test_vecs_w2v2 = scaling.transform(test_vecs_w2v)
classifier3 =  SVC(kernel = 'linear',
        cache_size= 1024, max_iter = 1500, verbose = True) 
classifier3.fit(train_vecs_w2v2, train_y2)
print (classifier3.get_params())
print( 'score : {}'.format(classifier3.score(test_vecs_w2v2, test_y2)))
print ('classification report')
print (classification_report(test_y2, classifier3.predict(test_vecs_w2v2)))
pickle.dump(classifier3,open(saveClassifierPath+'SVC_'+modelName, 'wb'))

#### 분류모델 : XGBOOST

In [ ]:
max_depth = 5
subsample = 0.7
colsample_bytree= 0.7
params ={
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "max_depth" : max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent":1,
    'eta':0.125,
    'nthread' : cores
    
    }
num_boost_round = 200
early_stopping_rounds = 10
test_size = 0.15

In [ ]:
dtrain = xgb.DMatrix(train_vecs_w2v, y_train)
dvalid = xgb.DMatrix(test_vecs_w2v, y_test)

In [ ]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals = watchlist, early_stopping_rounds = early_stopping_rounds, verbose_eval = True)
test_prediction = gbm.predict(xgb.DMatrix(test_vecs_w2v))
test_class = np.round(test_prediction)
print (accuracy_score(y_test, test_class))
print (classification_report(y_test, test_class))
gbm.save_model(saveClassifierPath+'XGBoost_'+modelName)

In [ ]:
%%time
Make_Roc_Curve(test_vecs_w2v, y_test, classifier, classifier2, classifier3, gbm)

In [ ]:
del classifier
del classifier2
del classifier3
del gbm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=1000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0,
          validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])

score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_1_'+modelName)

In [ ]:
plot_history(history)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_vecs_w2v, y_train, epochs=200, batch_size = 200000, verbose=0, validation_split=0.2,
          callbacks=[TQDMNotebookCallback(show_inner=False)])
score, acc = model.evaluate(test_vecs_w2v, y_test, verbose=0)
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)
model.save(saveClassifierPath+'NeuralNetwork_2_'+modelName)

In [ ]:
plot_history(history)